In [13]:
import os
import random
import torch
import torch.nn as nn
from PIL import Image
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import Subset
from torch.utils.data import random_split
from torch.utils.data import Dataset, DataLoader
from concurrent.futures import ThreadPoolExecutor
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(torch.cuda.get_device_name(device))

NVIDIA GeForce RTX 2060


In [3]:
class VERIDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []

        for img_name in os.listdir(data_dir):
            if img_name.endswith('.jpg'):
                self.image_paths.append(img_name)
                car_id = int(img_name.split('_')[0])
                self.labels.append(car_id)

        # Create a mapping for car IDs to indices
        self.id_to_indices = {}
        for idx, label in enumerate(self.labels):
            if label not in self.id_to_indices:
                self.id_to_indices[label] = []
            self.id_to_indices[label].append(idx)

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, index):
        img_path = os.path.join(self.data_dir, self.image_paths[index])
        label = self.labels[index]
        img = Image.open(img_path).convert('RGB')
        if self.transform:
            img = self.transform(img)
        return img, label

    def get_triplet(self):
        ids_list = random.sample(self.id_to_indices.keys(), 1)[0]
        anchor_idx = random.choice(self.id_to_indices[ids_list])
        positive_idx = random.choice(self.id_to_indices[ids_list])

        # Ensure negative is from a different ID
        negative_ids = list(self.id_to_indices.keys())
        negative_ids.remove(ids_list)
        negative_id = random.choice(negative_ids)
        negative_idx = random.choice(self.id_to_indices[negative_id])

        return anchor_idx, positive_idx, negative_idx

In [4]:
class TripletDataset(Dataset):
    def __init__(self, veri_dataset):
        self.veri_dataset = veri_dataset

    def __len__(self):
        return len(self.veri_dataset)

    def __getitem__(self, index):
        anchor_idx, positive_idx, negative_idx = self.veri_dataset.get_triplet()
        anchor, _ = self.veri_dataset[anchor_idx]
        positive, _ = self.veri_dataset[positive_idx]
        negative, _ = self.veri_dataset[negative_idx]
        return anchor, positive, negative

In [9]:
class TripletCNN(nn.Module):
    def __init__(self):
        super(TripletCNN, self).__init__()

        # First convolution layer: 7x7 kernel, stride 5, padding 3, 12 channels
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=7, stride=5, padding=3)
        self.pool1 = nn.MaxPool2d(kernel_size=2, stride=1)  # No downsampling here

        # Second convolution layer: 3x3 kernel, stride 1, padding 1, 24 channels
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=3, stride=1, padding=1)

        # Third convolution layer: 3x3 kernel, stride 1, padding 1, 32 channels (no downsampling)
        self.conv3 = nn.Conv2d(in_channels=24, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.pool2 = nn.MaxPool2d(kernel_size=2, stride=2)  # Reduces to 32x32

        # Fourth convolution layer: 3x3 kernel, stride 1, padding 1, 64 channels (no downsampling)
        self.conv4 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)

        # Adaptive pooling to prevent size issues
        self.global_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Flatten()

    def forward(self, x):
        x1 = F.relu(self.conv1(x))
        x2 = self.pool1(x1)

        x3 = F.relu(self.conv2(x2))
        x4 = F.relu(self.conv3(x3))
        x5 = self.pool2(x4)

        x6 = F.relu(self.conv4(x5))
        x7 = self.global_pool(x6)
        x8 = self.fc(x7)

        return x8



# Define Triplet Loss
class TripletLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(TripletLoss, self).__init__()
        self.margin = margin

    def forward(self, anchor, positive, negative):
        pos_dist = torch.nn.functional.pairwise_distance(anchor, positive)
        neg_dist = torch.nn.functional.pairwise_distance(anchor, negative)
        loss = torch.clamp(pos_dist - neg_dist + self.margin, min=0.0).mean()
        return loss



# Training loop
def train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=10):
    model.to(device)
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        for anchor, positive, negative in train_loader:
            anchor, positive, negative = anchor.to(device), positive.to(device), negative.to(device)
            optimizer.zero_grad()
            anchor_out = model(anchor)
            positive_out = model(positive)
            negative_out = model(negative)
            loss = criterion(anchor_out, positive_out, negative_out)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss / len(train_loader):.4f}")

In [ ]:

def main():
    data_dir = r'D:\Computer Vision\FYP\TASK 1\env\TrackNet-X\DataSet\VeRi\image_train'  # Change to your dataset path
    transform = transforms.Compose([
        transforms.Resize((128, 128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Load the complete dataset
    full_dataset = VERIDataset(data_dir, transform)

    # Split dataset into train and validation sets
    train_size = int(0.75 * len(full_dataset))
    val_size = len(full_dataset) - train_size
    train_indices, val_indices = torch.utils.data.random_split(range(len(full_dataset)), [train_size, val_size])

    # Create train and validation subsets
    train_dataset = Subset(full_dataset, train_indices)
    val_dataset = Subset(full_dataset, val_indices)

    # Wrap Subset objects with TripletDataset
    train_triplet_dataset = TripletDataset(full_dataset)
    val_triplet_dataset = TripletDataset(full_dataset)

    train_loader = DataLoader(train_triplet_dataset, batch_size=16, shuffle=True)
    val_loader = DataLoader(val_triplet_dataset, batch_size=16, shuffle=False)

    # Initialize the model, optimizer, and loss function
    model = TripletCNN()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = TripletLoss()

    # Set device and start training
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    train_model(model, train_loader, val_loader, optimizer, criterion, device, epochs=10)

if __name__ == "__main__":
    main()

C:\Users\ahmad\AppData\Local\Temp\ipykernel_19468\1456088092.py:33: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  ids_list = random.sample(self.id_to_indices.keys(), 1)[0]


Epoch 1/10, Train Loss: 0.3646
Epoch 2/10, Train Loss: 0.2342
Epoch 3/10, Train Loss: 0.2022
Epoch 4/10, Train Loss: 0.1829
Epoch 5/10, Train Loss: 0.1649
Epoch 6/10, Train Loss: 0.1570
Epoch 7/10, Train Loss: 0.1484
Epoch 8/10, Train Loss: 0.1393
Epoch 9/10, Train Loss: 0.1376
Epoch 10/10, Train Loss: 0.1250
